In [0]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [0]:
import os
import shutil
import random

base_dir = "/Volumes/users/kevin_romero/kaggle/x-ray-kaggle/chest-xray-pneumonia/chest_xray_aug"

train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")

classes = ["NORMAL", "PNEUMONIA"]
n_to_move = 100

for cls in classes:
    src_dir = os.path.join(train_dir, cls)
    dst_dir = os.path.join(val_dir, cls)

    os.makedirs(dst_dir, exist_ok=True)

    # list files in source class directory
    files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
    random.shuffle(files)

    # take up to n_to_move files (handles case where there are fewer than 100)
    to_move = files[:n_to_move]

    print(f"Moving {len(to_move)} files from {src_dir} to {dst_dir} ...")

    for fname in to_move:
        src = os.path.join(src_dir, fname)
        dst = os.path.join(dst_dir, fname)
        shutil.move(src, dst)

print("Done.")

In [0]:
import tensorflow as tf

volume = "/Volumes/users/kevin_romero/kaggle/x-ray-kaggle/chest-xray-pneumonia/chest_xray_aug/"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    volume + "train",
    label_mode="binary",
    image_size=(512, 512),
    color_mode="grayscale",
    batch_size=32,
    shuffle=True,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    volume + "val",
    label_mode="binary",
    image_size=(512, 512),
    color_mode="grayscale",
    batch_size=32,
    shuffle=False,
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    volume + "test",
    label_mode="binary",
    image_size=(512, 512),
    color_mode="grayscale",
    batch_size=32,
    shuffle=False,
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)

In [0]:
import torch
import torchvision.models as models
import torch.nn as nn

# 1. Load Pre-trained DenseNet121
model = models.densenet121(weights='IMAGENET1K_V1')

# 2. Modify the First Layer (Handle Grayscale)
# ImageNet expects 3 channels (RGB). X-rays are 1 channel (Gray).
# Option A: Repeat the X-ray channel 3 times (easy).
# Option B: Change the first conv layer (better for compute).
original_conv = model.features.conv0
model.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Copy weights from the original RGB filters (average them to making a grayscale filter)
with torch.no_grad():
    model.features.conv0.weight[:] = torch.mean(original_conv.weight, dim=1, keepdim=True)

# 3. Modify the Last Layer (Classifier)
num_features = model.classifier.in_features
# Change output to 1 (for binary: Sick vs Healthy) or N (for multi-class)
model.classifier = nn.Linear(num_features, 1) 

# 4. Loss & Optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3.0])) # Weight for imbalance
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [0]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

# X-rays are grayscale; map to 3 channels
def preprocess(x, y):
    x = tf.image.grayscale_to_rgb(x)
    x = tf.keras.applications.efficientnet.preprocess_input(x)
    return x, y

train_ds_pp = train_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
val_ds_pp   = val_ds.map(preprocess,   num_parallel_calls=AUTOTUNE)
test_ds_pp  = test_ds.map(preprocess,  num_parallel_calls=AUTOTUNE)

base_model = EfficientNetB0(
    include_top=False,
    input_shape=(512, 512, 3),
    weights="imagenet"
)
base_model.trainable = False  # first train only the head

inputs = layers.Input(shape=(512, 512, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc"), "accuracy"],
)

In [0]:
history = model.fit(
    train_ds_pp,
    validation_data=val_ds_pp,
    epochs=10,
)

# Fine‑tune
base_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc"), "accuracy"],
)
history_ft = model.fit(
    train_ds_pp,
    validation_data=val_ds_pp,
    epochs=5,
)

In [0]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

y_true = np.concatenate([y for _, y in val_ds_pp])
y_pred = model.predict(val_ds_pp).ravel()

print("AUC:", roc_auc_score(y_true, y_pred))

for t in [0.2, 0.3, 0.4, 0.5, 0.6]:
    y_hat = (y_pred >= t).astype(int)
    acc = accuracy_score(y_true, y_hat)
    tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
    sens = tp / (tp + fn + 1e-8)  # recall for positive
    spec = tn / (tn + fp + 1e-8)
    print(f"thr={t:.2f} acc={acc:.3f} sens={sens:.3f} spec={spec:.3f}")

In [0]:
# 1) Save full model (architecture + weights + optimizer)
model.save("/Workspace/Users/kevin.romero@databricks.com/Personal/databricks-free-edition-projects/DL/x-ray-classification/model.keras")

In [0]:
import tensorflow as tf

model = tf.keras.models.load_model("/Workspace/Users/kevin.romero@databricks.com/Personal/databricks-free-edition-projects/DL/x-ray-classification/model.keras")

In [0]:
from sklearn.metrics import confusion_matrix

best_thr = 0.6  # or whichever you like from the printout
y_hat = (y_pred >= best_thr).astype(int)
tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
print("TN, FP, FN, TP:", tn, fp, fn, tp)